# Setup All Workshop Dependencies
### Pip
### AWS CLI and AWS Python SDK (boto3)
### SageMaker
### PyTorch
### TensorFlow
### Hugging Face Transformers (BERT)
### TorchServe
### PyAthena
### Redshift
### AWS Data Wrangler
### StepFunctions
### Zip
### Matplotlib
### Seaborn

In [10]:
!git remote add vault https://github.com/joel-day/vault

error: remote vault already exists.


In [11]:
!git remote -v

origin	https://github.com/joel-day/vault (fetch)
origin	https://github.com/joel-day/vault (push)
vault	https://github.com/joel-day/vault (fetch)
vault	https://github.com/joel-day/vault (push)


# S3 is a key value store. You can share it with others, not stored on single server, great way to store anything!

## Use Boto3 to create new session

In [ ]:
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

## List everything in bucket

In [11]:
#!aws s3 ls s3://${bucket}/

## Make sure the we can access bucket by retrieving only the head

In [ ]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

## Use 'store' to move variables across sheets

#### Best for small things. Store output of Sheet1 then Load into Sheet2

In [13]:
#%store setup_s3_bucket_passed

# Use Amazon Rekognition to identify celebs in Photos and Movies 

## Photos

#### Import libraries 

In [1]:
import boto3
from IPython.display import HTML, display, Image as IImage
from PIL import Image, ImageDraw, ImageFont
import time
import os

#### Create Amazon Rekognition Instance

In [5]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

rekognition = boto3.client("rekognition")
s3 = boto3.client("s3")

#### Create a Temp Folder 

In [2]:
!mkdir -p ./tmp
temp_folder = "tmp/"

The syntax of the command is incorrect.


#### Uplaod image then display it for fun 

In [ ]:
imageName = "content-moderation/media/GrandTourjc.png"
display(IImage(url=s3.generate_presigned_url("get_object", Params={"Bucket": bucket, "Key": imageName})))

#### Run it!

In [ ]:
recognizeCelebritiesResponse = rekognition.recognize_celebrities(
    Image={
        "S3Object": {
            "Bucket": bucket,
            "Name": imageName,
        }
    }
)
display(recognizeCelebritiesResponse)

#### Show Image with Bounding Boxes Around Recognized Celebrities

In [ ]:
def drawBoundingBoxes(sourceImage, boxes):
    # blue, green, red, grey
    colors = ((255, 255, 255), (255, 255, 255), (76, 182, 252), (52, 194, 123))

    # Download image locally
    imageLocation = temp_folder + os.path.basename(sourceImage)
    s3.download_file(bucket, sourceImage, imageLocation)

    # Draws BB on Image
    bbImage = Image.open(imageLocation)
    draw = ImageDraw.Draw(bbImage)
    width, height = bbImage.size
    col = 0
    maxcol = len(colors)
    line = 3
    for box in boxes:
        x1 = int(box[1]["Left"] * width)
        y1 = int(box[1]["Top"] * height)
        x2 = int(box[1]["Left"] * width + box[1]["Width"] * width)
        y2 = int(box[1]["Top"] * height + box[1]["Height"] * height)

        draw.text((x1, y1), box[0], colors[col])
        for l in range(line):
            draw.rectangle((x1 - l, y1 - l, x2 + l, y2 + l), outline=colors[col])
        col = (col + 1) % maxcol

    imageFormat = "PNG"
    ext = sourceImage.lower()
    if ext.endswith("jpg") or ext.endswith("jpeg"):
        imageFormat = "JPEG"

    bbImage.save(imageLocation, format=imageFormat)

    display(bbImage)

In [ ]:
boxes = []
celebrities = recognizeCelebritiesResponse["CelebrityFaces"]
for celebrity in celebrities:
    boxes.append((celebrity["Name"], celebrity["Face"]["BoundingBox"]))

drawBoundingBoxes(imageName, boxes)

## Videos

#### Load video (may be wrong)

In [ ]:
videoName = "content-moderation/media/GrandTour720.mp4"
strDetail = "Celebrites detected in video<br>=======================================<br>"
strOverall = "Celebrities in the overall video:<br>=======================================<br>"

In [ ]:
s3FilePrefix = "https://s3.amazonaws.com"
if not region == "us-east-1":
    s3FilePrefix = "https://s3-{}.amazonaws.com".format(region)

s3VideoUrl = "{0}/{1}/{2}".format(s3FilePrefix, bucket, videoName)

videoTag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(
    s3VideoUrl
)

videoui = "<table><tr><td style='vertical-align: top'>{}</td></tr><tr><td>{}</td></tr></table>".format(
    videoTag, strDetail
)

display(HTML(videoui))

#### Run it!

In [ ]:
startCelebrityRekognition = rekognition.start_celebrity_recognition(
    Video={
        "S3Object": {
            "Bucket": bucket,
            "Name": videoName,
        }
    },
)

celebrityJobId = startCelebrityRekognition["JobId"]
display("Job Id: {0}".format(celebrityJobId))

#### Wait for Celebrity Rekognition Job to Complete
###### In production use cases, you would usually use StepFucntion or SNS topic to get notified when job is complete.

In [ ]:
%%time

getCelebrityRecognition = rekognition.get_celebrity_recognition(JobId=celebrityJobId, SortBy="TIMESTAMP")

while getCelebrityRecognition["JobStatus"] == "IN_PROGRESS":
    time.sleep(5)
    print(".", end="")

    getCelebrityRecognition = rekognition.get_celebrity_recognition(JobId=celebrityJobId, SortBy="TIMESTAMP")

display(getCelebrityRecognition["JobStatus"])

In [ ]:
display(getCelebrityRecognition)

#### Display Names of Recognized Celebrities in the Video
#### Display timestamps and celebrites detected at that time.

In [ ]:
theCelebs = {}

# Celebrities detected in each frame
for celebrity in getCelebrityRecognition["Celebrities"]:
    if "Celebrity" in celebrity:
        cconfidence = celebrity["Celebrity"]["Confidence"]
        if cconfidence > 95:
            ts = celebrity["Timestamp"]
            cname = celebrity["Celebrity"]["Name"]
            strDetail = strDetail + "At {} ms: {} (Confidence: {})<br>".format(ts, cname, round(cconfidence, 2))
            if not cname in theCelebs:
                theCelebs[cname] = cname


# Unique faces detected in video
for theCeleb in theCelebs:
    strOverall = strOverall + "Name: {}<br>".format(theCeleb)

# Display results
display(HTML(strOverall))

# Content Moderation API

In [ ]:
import boto3
from IPython.display import Image as IImage, display
from IPython.display import HTML, display
from PIL import Image, ImageDraw, ImageFont
import time
import os

import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

rekognition = boto3.client("rekognition")
s3 = boto3.client("s3")

#### Load Image

In [ ]:
imageName = "content-moderation/media/weapon.png"
display(IImage(url=s3.generate_presigned_url("get_object", Params={"Bucket": bucket, "Key": imageName})))

#### Run it!

In [8]:
detectModerationLabelsResponse = rekognition.detect_moderation_labels(
    Image={
        "S3Object": {
            "Bucket": bucket,
            "Name": imageName,
        }
    }
)

display(detectModerationLabelsResponse)

#### Display list of detected moderation labels

In [ ]:
for label in detectModerationLabelsResponse["ModerationLabels"]:
    print("- {} (Confidence: {})".format(label["Name"], label["Confidence"]))
    print("  - Parent: {}".format(label["ParentName"]))